# Clap Detection Model Training

This notebook trains a model to detect clapping gestures from accelerometer data.

Upload a `clapping-data.json` file in the following format, and then click 'run all' to train the model:
```typescript
// [x, y, z, magnitude]
type Sample = [number, number, number, number];
interface ClappingData {
  '0': Array<Sample>, // Not clapping
  '1': Array<Sample>, // Clapping
}
```

In [39]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Concatenate, GlobalAveragePooling1D, Reshape, multiply
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

## Data and Model Constants

In [40]:
WINDOW_SIZE = 30
NUM_CHANNELS = 4  # x, y, z, magnitude
INPUT_SHAPE = (WINDOW_SIZE, NUM_CHANNELS)
RECORD_STRIDE = 20

## Data Preparation

In [41]:
def prepare_data(gestures):
    """Prepares data for training."""
    inputs = []
    outputs = []

    def process_gesture(gesture_samples, label):
        if len(gesture_samples) < WINDOW_SIZE:
            return
        for i in range(0, len(gesture_samples) - WINDOW_SIZE + 1, RECORD_STRIDE):
            window = gesture_samples[i : i + WINDOW_SIZE]
            inputs.append(window)
            outputs.append([label])

    process_gesture(gestures.get("0", []), 0)
    process_gesture(gestures.get("1", []), 1)

    if not inputs:
        return None, None

    input_tensor = np.array(inputs, dtype=np.float32)
    output_tensor = np.array(outputs, dtype=np.float32)

    return input_tensor, output_tensor

## Data Augmentation

In [42]:
def augment_data(data, noise_level=0.01):
    """Adds random noise to the data for augmentation."""
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

## Squeeze-and-Excitation Block

In [43]:
def se_block(input_tensor, filters, ratio=8):
    """Creates a Squeeze-and-Excitation block."""
    se_shape = (1, filters)

    se = GlobalAveragePooling1D()(input_tensor)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', use_bias=False)(se)

    return multiply([input_tensor, se])

## Model Training Function

In [44]:
def train_model(data_path="clapping-data.json", tflite_path="clap_model.tflite", use_se=False, num_conv_blocks=2,
                    filters=32, kernel_size=5, dense_units=16, dropout_rate=0.25):
    """Loads data, defines, trains, and exports the model."""
    print(f"Loading data from {data_path}...")
    try:
        with open(data_path, "r") as f:
            gestures = json.load(f)
    except FileNotFoundError:
        print(f"Error: Data file not found at {data_path}")
        print("Please export the data from the HTML file and place it in the same directory.")
        return

    print("Preparing training data...")
    input_tensor, output_tensor = prepare_data(gestures)

    if input_tensor is None:
        print("Not enough data to form a complete window for training.")
        return

    print(f"Prepared {len(input_tensor)} total samples.")

    print("Splitting data into training and validation sets...")
    x_train, x_val, y_train, y_val = train_test_split(
        input_tensor, output_tensor, test_size=0.2, random_state=42, stratify=output_tensor
    )

    print(f"Training samples: {len(x_train)}, Validation samples: {len(x_val)}")

    print("Augmenting training data...")
    x_train_aug = augment_data(x_train)

    print(f"CNN Hyperparameters: num_conv_blocks={num_conv_blocks}, filters={filters}, kernel_size={kernel_size}, "
          f"dense_units={dense_units}, dropout_rate={dropout_rate}")

    input_layer = Input(shape=INPUT_SHAPE)

    # CNN Blocks
    x = input_layer
    current_filters = filters
    for i in range(1, num_conv_blocks):
        x = Conv1D(filters=current_filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
        if use_se:
            x = se_block(x, current_filters)
        x = MaxPooling1D(pool_size=2)(x)
        current_filters *= 2

    x = Flatten()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    print("\nTraining model...")
    model.fit(x_train_aug, y_train, epochs=50, shuffle=True,
              validation_data=(x_val, y_val),
              callbacks=[early_stopping])

    print("\nTraining complete! Converting to TensorFlow Lite...")

    # Convert the model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the model
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)

    print(f"Model saved successfully to {tflite_path}")

## Configuration and Training

Here you can configure the training parameters and run the training process.

In [45]:
# --- Configuration ---
DATA_PATH = "clapping-data.json"
OUTPUT_PATH = "clap_model.tflite"

# Architecture
USE_SE = True

# Standard CNN Hyperparameters
NUM_CONV_BLOCKS = 2
FILTERS = 32
KERNEL_SIZE = 5
DENSE_UNITS = 16
DROPOUT_RATE = 0.25

# --- Train ---
train_model(
    data_path=DATA_PATH,
    tflite_path=OUTPUT_PATH,
    use_se=USE_SE,
    num_conv_blocks=NUM_CONV_BLOCKS,
    filters=FILTERS,
    kernel_size=KERNEL_SIZE,
    dense_units=DENSE_UNITS,
    dropout_rate=DROPOUT_RATE
)

Loading data from clapping-data.json...
Preparing training data...
Prepared 1091 total samples.
Splitting data into training and validation sets...
Training samples: 872, Validation samples: 219
Augmenting training data...
CNN Hyperparameters: num_conv_blocks=2, filters=32, kernel_size=5, dense_units=16, dropout_rate=0.25


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 30, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 30, 32)    │        672 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ conv1d_4[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_4 (Reshape) │ (None, 1, 32)     │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1, 4)      │        128 │ reshape_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 1, 32)     │        128 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_4          │ (None, 30, 32)    │          0 │ conv1d_4[0][0],   │
│ (Multiply)          │                   │            │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 15, 32)    │          0 │ multiply_4[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 480)       │          0 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 16)        │      7,696 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16)        │          0 │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 1)         │         17 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,641 (33.75 KB)

 Trainable params: 8,641 (33.75 KB)

 Non-trainable params: 0 (0.00 B)


Training model...
Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.5010 - loss: 0.6919 - val_accuracy: 0.7078 - val_loss: 0.6505
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6376 - loss: 0.6524 - val_accuracy: 0.7123 - val_loss: 0.6045
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6457 - loss: 0.6258 - val_accuracy: 0.7078 - val_loss: 0.5839
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7296 - loss: 0.5815 - val_accuracy: 0.7443 - val_loss: 0.5521
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7341 - loss: 0.5573 - val_accuracy: 0.7763 - val_loss: 0.5114
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7736 - loss: 0.5293 - val_accuracy: 0.7991 - val_loss: 0.4899
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7856 - loss: 0.5226 - val_accuracy: 0.8402 - val_loss: 0.4306
Epoch 8/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8147 - loss: 0.4835 - val_accuracy